In [8]:
import selenium
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from dateutil import parser
from random import randint

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Scrapping

In [10]:
symbols = ["TRGU", "CAMP", "WMUU", "MAIN", "CEKA", "CSRA", "AISA", "HOKI"]

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    # Handdle: IndexError: list index out of range
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    df_c.drop(len(df_c) -1)
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

symbol TRGU date_elements 161 dates 160 comment_elements 161 comments 160 
symbol CAMP date_elements 101 dates 100 comment_elements 101 comments 100 
symbol WMUU date_elements 241 dates 240 comment_elements 261 comments 240 
symbol MAIN date_elements 101 dates 100 comment_elements 101 comments 100 
symbol CEKA date_elements 41 dates 40 comment_elements 41 comments 40 
symbol CSRA date_elements 121 dates 120 comment_elements 121 comments 120 
symbol AISA date_elements 441 dates 440 comment_elements 441 comments 440 
symbol HOKI date_elements 221 dates 220 comment_elements 221 comments 220 


In [15]:
df.to_csv('scraping_arya.csv', index=False)